<a href="https://colab.research.google.com/github/EziamaUgonna/Bayesian_analysis-/blob/master/Dessert_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Install packages
#!pip install tensorflow==2.0.0-beta1
#!pip install tensorflow_addons as tfa

In [0]:
try:
  # % tensorflow_version only exist in colab
  %tensorflow_version 2.
except Exception:
   pass

`%tensorflow_version` only switches the major version: `1.x` or `2.x`.
You set: `2.`. This will be interpreted as: `2.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


In [0]:
# Packages
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
#import vaex
#from sklearn.dummy import DummyRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
import tensorflow as tf
#import tensorflow_addons as tfa
import tensorflow_probability as tfp
tfd = tfp.distributions
from tensorflow_probability.python.math import random_rademacher
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras.layers import ReLU, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Settings
sns.set()
%config InlineBackend.figure_format = 'svg'
np.random.seed(12345)
#tf.random.set_seed(12345)
tf.__version__

'2.0.0'

In [0]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!ls "/content/drive/My Drive"

In [0]:
# Read in the dataset
"""df = pd.read_csv('/content/gdrive/My Drive/work2.csv').astype(np.float32)
df.columns = ['RxDevice', 'FileID', 'TxDevice', 'Gentime', 'TxRandom', 'MsgCount',
       'Dsecond', 'Latitude', 'Longitude', 'Elevation', 'Speed', 'Heading',
       'Ax', 'Ay', 'Az', 'Yawrate', 'PathCount', 'RadiusOfCurve', 'Confidence']
change = df.query('Speed>0').sample(frac = .1).index
df.loc[change, 'Speed'] = 0
df.loc[change, 'Class'] = 0
df.to_csv('work2.csv', header = True, index =False)
df.isnull().values.any() , df.isnull().sum(),df.shape"""

In [0]:
df1 = pd.read_csv('/content/drive/My Drive/Sensor1_values_bias_0_1_dur_10_dep.csv').astype(np.float32)
df2 = pd.read_csv('/content/drive/My Drive/Ground_truth_bias_0_1_dur_10_sensor1_dep.csv')

NameError: ignored

In [0]:
#X = df.iloc[:,:-1].values
#y = df.iloc[:,-1].values
X = df1.values
y = df2.values
X.shape, y.shape

In [0]:
X_train_full, X_test,y_train_full,y_test = train_test_split(X, y, test_size=0.2, random_state=200)
X_train, X_valid, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=1)
#reshape y-data to become column vector 
y_train = np.reshape(y_train,[-1,1])
y_val   = np.reshape(y_val,[-1,1])
y_test  = np.reshape(y_test,[-1,1])

In [0]:
# Standardize the dataset 
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

In [0]:
# Vanilla Neural Network
# Batch size
BATCH_SIZE = 1024

# Number of training epochs
EPOCHS = 100

# Learning rate
L_RATE = 1e-4

# Proportion of samples to hold out
VAL_SPLIT = 0.2
training_steps = 2000


In [0]:
train_data = tf.data.Dataset.from_tensor_slices((X_train_scaled, y_train)).shuffle(1000).repeat(3).batch(7)
valid_data = tf.data.Dataset.from_tensor_slices((X_valid_scaled,y_val)).shuffle(1000).repeat(3).batch(7)
test_data = tf.data.Dataset.from_tensor_slices((X_test_scaled,y_test)).shuffle(1000).repeat(3).batch(7)

In [0]:
# Xavier initializer
def xavier(shape):
    return tf.random.truncated_normal(
        shape, 
        mean=0.0,
        stddev=np.sqrt(2/sum(shape)))

In [0]:
class BayesianDenseLayer(tf.keras.Model):
        
    def __init__(self, d_in, d_out, name=None):
        
        super(BayesianDenseLayer, self).__init__(name=name)
        self.d_in = d_in
        self.d_out = d_out
        
        self.w_loc = tf.Variable(xavier([d_in, d_out]), name='w_loc')
        self.w_std = tf.Variable(xavier([d_in, d_out])-6.0, name='w_std')
        self.b_loc = tf.Variable(xavier([1, d_out]), name='b_loc')
        self.b_std = tf.Variable(xavier([1, d_out])-6.0, name='b_std')
    
    
    def call(self, x, sampling=True):
        """Perform the forward pass"""
        
        if sampling:
        
            # Flipout-estimated weight samples
            s = random_rademacher(tf.shape(x))
            r = random_rademacher([x.shape[0], self.d_out])
            w_samples = tf.nn.softplus(self.w_std)*tf.random.normal([self.d_in, self.d_out])
            w_perturbations = r*tf.matmul(x*s, w_samples)
            w_outputs = tf.matmul(x, self.w_loc) + w_perturbations
            
            # Flipout-estimated bias samples
            r = random_rademacher([x.shape[0], self.d_out])
            b_samples = tf.nn.softplus(self.b_std)*tf.random.normal([self.d_out])
            b_outputs = self.b_loc + r*b_samples
            
            return w_outputs + b_outputs
        
        else:
            return x @ self.w_loc + self.b_loc
    
    
    @property
    def losses(self):
        """Sum of the KL divergences between priors + posteriors"""
        weight = tfd.Bernoulli(self.w_loc, tf.nn.softplus(self.w_std))
        bias = tfd.Bernoulli(self.b_loc, tf.nn.softplus(self.b_std))
        prior = tfd.Bernoulli(0, 1)
        return (tf.reduce_sum(tfd.kl_divergence(weight, prior)) +
                tf.reduce_sum(tfd.kl_divergence(bias, prior)))

In [0]:
class BayesianDenseNetwork(tf.keras.Model):
    
    def __init__(self, dims, name=None):
        
        super(BayesianDenseNetwork, self).__init__(name=name)
        
        self.steps = []
        self.acts = []
        for i in range(len(dims)-1):
            self.steps += [BayesianDenseLayer(dims[i], dims[i+1])]
            self.acts += [tf.nn.relu]
            
        self.acts[-1] = lambda x: x
        
    
    def call(self, x, sampling=True):
        """Perform the forward pass"""

        for i in range(len(self.steps)):
            x = self.steps[i](x, sampling=sampling)
            x = self.acts[i](x)
            
        return x
    
    
    @property
    def losses(self):
        """Sum of the KL divergences between priors + posteriors"""
        return tf.reduce_sum([s.losses for s in self.steps])

In [0]:
class BayesianDenseClassifier(tf.keras.Model):
           
    def __init__(self, dims, name=None):
        
        super(BayesianDenseClassifier, self).__init__(name=name)
        
        # Multilayer fully-connected neural network to predict mean
        self.net = BayesianDenseNetwork(dims) #net to predict categorical probs 
        
       
    
    def call(self, x, sampling=True):
        """Perform forward pass, predicting both means + stds"""
        logits = self.net(x, sampling = sampling)
        return tfd.Bernoulli(logits=logits)
        
       
    
    def log_likelihood(self, x, y, sampling=True):
        """Compute the log likelihood of y given x"""
        return self.call(x, sampling= sampling).log_prob(y)
   
    @tf.function
    def sample(self, x):
        """Draw one sample from the predictive distribution"""
       
        preds = self.call(x)
        return tfd.Bernoulli(preds[:,0]).sample()
        #return tfd.Bernoulli(preds[:,0], preds[:,1]).sample()
    
    
    def samples(self, x, n_samples=1):
        """Draw multiple samples from the predictive distribution"""
        samples = np.zeros((x.shape[0], n_samples))
        for i in range(n_samples):
            samples[:,i] = self.sample(x)
        return samples
        
    @property
    def losses(self):
        """Sum of the KL divergences between priors + posteriors"""
                
        # Loss due to network weights
        #net_loss = self.loc_net.losses

        # Loss due to std deviation parameter
        #posterior = tfd.Gamma(self.std_alpha, self.std_beta)
        #prior = tfd.Gamma(10.0, 10.0)
        #std_loss = tfd.kl_divergence(posterior, prior)

        # Return the sum of both
        #return net_loss + std_loss
        return self.net.losses
model1 = BayesianDenseClassifier([4, 32,16,2])

NameError: ignored

In [0]:

loss_func = tf.keras.losses.BinaryCrossentropy
fpr = tf.keras.metrics.FalsePositives()
Tpr = tf.keras.metrics.TruePositives()
AUC = tf.keras.metrics.AUC()
#test_accuracy = tf.keras.metrics.binary_accuracy(y_train, y_pred)
#F1_score = tfa.metrics.f_scores.F1Score()
# Adam optimizer
optimizer = tf.keras.optimizers.Adam(lr=L_RATE)


NameError: ignored

In [0]:
N = X_train.shape[0]
@tf.function
def train_step(x_data, y_data):
    with tf.GradientTape() as tape:
        log_likelihoods = model1.log_likelihood(x_data, y_data)
        kl_loss = model1.losses
        elbo_loss = kl_loss/N - tf.reduce_mean(log_likelihoods)
    gradients = tape.gradient(elbo_loss, model1.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model1.trainable_variables))
    return elbo_loss

In [0]:
# Fit the model
elbo1 = np.zeros(EPOCHS)
#mae1 = np.zeros(EPOCHS)
for epoch in range(EPOCHS):
    
    # Update weights each batch
    for x_data, y_data in train_data:
        elbo1[epoch] += train_step(x_data, y_data)
        
    # Evaluate performance on validation data
    for x_data, y_data in test_data:
        y_pred = model1(x_data).mode()
        #mae1[epoch] = mean_absolute_error(y_pred, y_data)
        test_accuracy(y_pred, y_data)
        print("step: %i, loss: %f, accuraccy: :%f" %(step, loss, acc))

In [0]:
# Plot the ELBO loss
plt.plot(elbo1)
plt.xlabel('Epoch')
plt.ylabel('ELBO Loss')
plt.show()

In [0]:
# Plot validation error over training
plt.plot(mae1)
plt.xlabel('Epoch')
plt.ylabel('Mean Absolute Error')
plt.show()